In [1]:
# imports
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import json
import yfinance as yf
import datetime

/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
# read purchases and sales data
purchases = pd.read_excel('data/purchases_history.xlsx')
sales = pd.read_excel('data/sales_history.xlsx')

In [3]:
tickers = purchases['asset'].unique()
for t in sales['asset'].unique():
    if t not in tickers:
        tickers = np.append(tickers, t)

In [4]:
# establish the portfolio quantities dictionary
# dict = {date: [{<ticker>, <quantity>, <price>, <value>}, ...]}
min_date = min(purchases['actual_purchase_date'].min(), sales['actual_sale_date'].min())
max_date = pd.Timestamp(datetime.datetime.today())
dates = pd.date_range(min_date, max_date, freq='D')

In [5]:
portfolio = {date: [] for date in dates}
# build out quantities by adding purchases
for i in range(len(purchases)):
    date = purchases.iloc[i]['actual_purchase_date']
    ticker = purchases.iloc[i]['asset']
    quantity = purchases.iloc[i]['purchase_qty']
    # append this to all dates from the purchase date to the end of the date range in portfolio
    for d in dates[dates.get_loc(date):]:
        if ticker in [x['ticker'] for x in portfolio[d]]:
            for x in portfolio[d]:
                if x['ticker'] == ticker:
                    x['quantity'] += quantity
        else:
            portfolio[d].append({'ticker': ticker, 'quantity': quantity})

In [6]:
# now subtract sales
for i in range(len(sales)):
    date = sales.iloc[i]['actual_sale_date']
    ticker = sales.iloc[i]['asset']
    quantity = sales.iloc[i]['sale_qty']
    # support lazy quantities of zero
    flat = False
    if quantity == 0:
        flat = True
    # for all dates from the sale date to the end of the date range in the port
    for d in dates[dates.get_loc(date):]:
        # check if this ticker is in the portfolio
        holdings = portfolio[d]
        if ticker in [i['ticker'] for i in holdings]:
            # update the quantity for this ticker holding
            for h in holdings:
                if h['ticker'] == ticker:
                    if flat:
                        h['quantity'] = 0
                    else:
                        h['quantity'] += quantity
                    break
            portfolio[d] = holdings
        else:
            raise Exception('Sale of {} on {} not in portfolio'.format(ticker, d))

In [7]:
# clean by removing any holdings with quantity zero
for d in portfolio:
    portfolio[d] = [h for h in portfolio[d] if h['quantity'] != 0]

In [8]:
# check
portfolio[dates[-1]]

[{'ticker': 'MTN', 'quantity': 34},
 {'ticker': 'CLBT', 'quantity': 4766},
 {'ticker': 'BLBD', 'quantity': 1561},
 {'ticker': 'BELFB', 'quantity': 452},
 {'ticker': 'ULCC', 'quantity': 1439},
 {'ticker': 'GXO', 'quantity': 286},
 {'ticker': 'NFE', 'quantity': 586},
 {'ticker': 'OCSL', 'quantity': 2188}]

In [9]:
hist_data_store = {x: None for x in tickers}
# use yfinance to populate the historical data dictionary
for ticker in tickers:
    start = purchases[purchases['asset'] == ticker]['actual_purchase_date'].min()
    end = max_date
    try:
        df = yf.download(ticker, start=start, end=end)
        if len(df) == 0:
            raise Exception()
        hist_data_store[ticker] = df
    except:
        print('Error downloading data for {}'.format(ticker))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['SCPL']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Error downloading data for SCPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ECOM']: Exception('%ticker%: No timezone found, symbol may be delisted')


Error downloading data for ECOM


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MAXR']: Exception('%ticker%: No timezone found, symbol may be delisted')


Error downloading data for MAXR


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [10]:
# reconcile with backup data
backup = pd.read_excel('data/backup_data_2.xlsx')
for ticker in tickers:
    if hist_data_store[ticker] is None:
        # start = purchases[purchases['asset'] == ticker]['actual_purchase_date'].min()
        # end = max_date
        try:
            df = pd.DataFrame(index=backup['Date'], columns=['Adj Close'])
            df['Adj Close'] = backup[ticker].values
            df = df.dropna()
            hist_data_store[ticker] = df
        except:
            print('No backup data found for {}'.format(ticker))

In [11]:
# helpful methods
def get_price(date, df):
    if df is None or len(df) == 0:
        return 0
    try:
        return df.loc[date]['Adj Close']
    except:
        # get the closest price to the date
        closest_date = df.index[0]
        for d in df.index:
            if abs((d - date).days) < abs((closest_date - date).days):
                closest_date = d
        return df.loc[closest_date]['Adj Close']

If we know ending cash and know all historical transactions, we can back-calculate starting cash and back-fill the portfolio

In [12]:
ENDCASH=21210
# generate a cash value dataframe

cashin = 0
cashout = 0

for i in range(len(purchases)):
    date = purchases.iloc[i]['actual_purchase_date']
    ticker = purchases.iloc[i]['asset']
    quantity = purchases.iloc[i]['purchase_qty']
    price = get_price(date, hist_data_store[ticker])
    cashout += quantity * price

for i in range(len(sales)):
    date = sales.iloc[i]['actual_sale_date']
    ticker = sales.iloc[i]['asset']
    quantity = sales.iloc[i]['sale_qty']
    price = get_price(date, hist_data_store[ticker])
    cashin += -quantity * price

STARTCASH = ENDCASH - cashin + cashout

cash_value = pd.DataFrame(index=dates, columns=['USD'])
cash_value['USD'] = STARTCASH

for i in range(len(purchases)):
    date = purchases.iloc[i]['actual_purchase_date']
    ticker = purchases.iloc[i]['asset']
    quantity = purchases.iloc[i]['purchase_qty']
    price = get_price(date, hist_data_store[ticker])
    # append this to all dates from the purchase date to the end of the date range in portfolio
    for d in dates[dates.get_loc(date):]:
        cash_value.loc[d]['USD'] -= quantity * price

for i in range(len(sales)):
    date = sales.iloc[i]['actual_sale_date']
    ticker = sales.iloc[i]['asset']
    quantity = sales.iloc[i]['sale_qty']
    price = get_price(date, hist_data_store[ticker])
    # append this to all dates from the purchase date to the end of the date range in portfolio
    for d in dates[dates.get_loc(date):]:
        cash_value.loc[d]['USD'] += -quantity * price

cash_value.tail()

,USD
2024-01-25,21210.0
2024-01-26,21210.0
2024-01-27,21210.0
2024-01-28,21210.0
2024-01-29,21210.0


In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cash_value.index, y=cash_value['USD'], name='Cash Value'))
fig.update_layout(title='Cash Value Over Time')
fig.show()

In [14]:
# add cash values to the portfolio dict
for d in dates:
    portfolio[d].append({'ticker': 'USD', 'quantity': cash_value.loc[d]['USD'], 'price': 1, 'value': cash_value.loc[d]['USD']})

In [15]:
# build portfolio value
for d in portfolio:
    for h in portfolio[d]:
        ticker = h['ticker']
        if ticker == 'USD':
            continue
        quantity = h['quantity']
        price = get_price(d, hist_data_store[ticker])
        h['price'] = price
        h['value'] = price * quantity

In [16]:
# generate a portfolio value dataframe
portfolio_value = pd.DataFrame(index=dates, columns=['value'])
for d in portfolio_value.index:
    portfolio_value.loc[d]['value'] = sum([h['value'] for h in portfolio[d]])

portfolio_value.tail()

,value
2024-01-25,235830.514121
2024-01-26,237288.253916
2024-01-27,237288.253916
2024-01-28,241413.844757
2024-01-29,241413.844757


In [17]:
# plot the portfolio value over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio_value.index, y=portfolio_value['value'], name='Portfolio Value'))
fig.update_traces(line=dict(color='black'))
fig.update_layout(title='Portfolio Value Over Time', yaxis_title='Value (USD)')
fig.update_layout(font_family='Courier New, monospace', font_size=18, font_color='black')
fig.show()

In [26]:
# output the portfolio value to an excel
portfolio_value.to_excel('data/portfolio_value.xlsx')